In [1]:
import pandas as pd
import re

In [95]:
draws = pd.read_csv('eoi.csv')
draws['date'] = draws.date.astype('datetime64[ns]')
draws = draws.sort_values(by=['date'], ascending=False).reset_index(drop=True)
draws = draws.reset_index()
draws.columns = ['draw_id', 'date', 'draw_type', 'score', 'invitations', 'nocs']
draws.draw_id = draws.draw_id + 1

In [96]:
nocs_draws_obj = {'noc_id': [], 'draw_id': []}
for i in range (len(draws)):
    nocs_list =  [x.strip() for x in draws.iloc[i].nocs.split(',')]
    draw_id = draws.iloc[i].draw_id
    nocs_draws_obj['noc_id'] = nocs_draws_obj['noc_id'] + nocs_list
    nocs_draws_obj['draw_id'] = nocs_draws_obj['draw_id'] + [draw_id]*len(nocs_list)

In [97]:
df = pd.read_csv('./noc_list_raw.csv')
df['noc'] = df['noc'].astype('str')
df['noc'] = df['noc'].apply(lambda x: '{0:0>4}'.format(x))
pd.set_option('display.max_rows',200)

In [98]:
nocs = df
nocs.columns = ['noc_id', 'title', 'skill_level', '2019_est_employments', '2019_wage_est', 'job_outlook', 'job_openings', 'soft_skills']
nocs_draws= pd.DataFrame(nocs_draws_obj)

In [103]:
z = nocs.merge(nocs_draws, on='noc_id', how='left').merge(draws, how='left')[['noc_id','date', 'draw_type']]
z['draw_date_type'] = z['date'].astype('str')+'_'+z['draw_type']
z = z.drop_duplicates().sort_values(by='noc_id')
x=pd.get_dummies(z['draw_date_type'])
j=pd.concat([z,x], axis=1)
j = j.drop(['draw_date_type'], axis=1)
k = j.groupby('noc_id')
final = nocs.merge(k.sum(), on='noc_id')
final = final.drop(['soft_skills', '2019_wage_est'], axis=1)
final['total'] = final.filter(regex='2019-|2020-|2021.*').sum(axis=1)
final.update(final.filter(regex='2021-|2020-|2019-.*').replace(to_replace=[1,0],value=['YES', 'NO']))
columns = final.columns.to_list()
col=columns[0:6]+[columns[-1]]+columns[6:-1]
final[col]

,noc_id,title,skill_level,2019_est_employments,job_outlook,job_openings,total,2019-09-25_Express Entry,2019-09-25_Occupations In-Demand,2019-10-02_Express Entry,...,2020-12-17_Occupations In-Demand,2021-01-07_Express Entry,2021-01-07_Occupations In-Demand,2021-01-21_Express Entry,2021-01-21_Occupations In-Demand,2021-02-11_Express Entry,2021-02-11_Occupations In-Demand,2021-02-25_Express Entry,2021-03-11_Express Entry,2021-03-11_Occupations In-Demand
0,0011,Legislators,Mgt,450,Fair,140,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
1,0012,Senior government managers and officials,Mgt,690,Good,170,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
2,0013,"Senior managers - financial, communications an...",Mgt,1020,Good,240,45,YES,YES,YES,...,YES,YES,YES,YES,YES,YES,YES,NO,NO,NO
3,0014,"Senior managers - health, education, social an...",Mgt,890,Good,240,36,NO,NO,YES,...,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES
4,0015,"Senior managers - trade, broadcasting and othe...",Mgt,850,Fair,250,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,9612,Labourers in metal fabrication,LHS,340,Fair,50,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
433,9613,Labourers in chemical products processing and ...,LHS,160,Fair,30,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
434,9614,"Labourers in wood, pulp and paper processing",LHS,190,Fair,30,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
435,9617,"Labourers in food, beverage and associated pro...",LHS,1710,Fair,310,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


In [89]:
columns = final.columns.to_list()

In [92]:
col=columns[0:6]+[columns[-1]]+columns[6:-1]

In [78]:
# import seaborn as sns
# corr = final.corr()
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=1000),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# );

In [93]:
len(columns)

64

In [94]:
len(col)

64